In [2]:
import pandas as pd 
import numpy as np 
import prophet
import holidays 
import plotly 
import streamlit 

d:\Atm Demand Forecasting\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data = pd.read_csv('dataset.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.head(2)

,ATM_ID,Date,Day_of_Week,Time_of_Day,Total_Withdrawals,Total_Deposits,Location_Type,Holiday_Flag,Special_Event_Flag,Previous_Day_Cash_Level,Weather_Condition,Nearby_Competitor_ATMs,Cash_Demand_Next_Day
0,ATM_0041,2022-04-25,Monday,Morning,57450,9308,Standalone,0,0,112953,Rainy,5,44165
1,ATM_0007,2023-11-24,Friday,Morning,72845,17896,Supermarket,0,0,115348,Clear,0,41249


In [18]:
daily_demand = data.groupby(['Date','ATM_ID']).agg(
    y = ('Total_Withdrawals', sum),
    Holiday_Flag = ('Holiday_Flag', max)
).reset_index()
daily_demand

C:\Users\DELL\AppData\Local\Temp\ipykernel_22068\1798609538.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  daily_demand = data.groupby(['Date','ATM_ID']).agg(
C:\Users\DELL\AppData\Local\Temp\ipykernel_22068\1798609538.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  daily_demand = data.groupby(['Date','ATM_ID']).agg(


,Date,ATM_ID,y,Holiday_Flag
0,2022-01-01,ATM_0003,28640,1
1,2022-01-01,ATM_0008,41577,1
2,2022-01-01,ATM_0012,45965,1
3,2022-01-01,ATM_0018,50215,1
4,2022-01-01,ATM_0037,88394,1
...,...,...,...,...
5218,2024-01-01,ATM_0016,46297,1
5219,2024-01-01,ATM_0024,39808,1
5220,2024-01-01,ATM_0029,39006,1
5221,2024-01-01,ATM_0039,60128,1


In [19]:
us_holidays = holidays.US( years = daily_demand['Date'].dt.year.unique())

holidays_df = pd.DataFrame({
    'ds' : [date for date in us_holidays.keys()],
    'holiday' : ['US_Bank_Holiday'] * len(us_holidays)
})

holidays_df['ds'] = pd.to_datetime(holidays_df['ds'])

In [26]:
#TIME SERIES MODELLING 
from prophet import Prophet 

ATM_TO_FORECAST = 'ATM_0007'
atm_data = daily_demand[daily_demand['ATM_ID'] == ATM_TO_FORECAST].copy()
#Rename for Prophet: Date -> ds and Total_Withdrawals -> y
atm_data = atm_data.rename(columns = {'Date': 'ds'})[['ds','y','Holiday_Flag']].dropna()


In [46]:
m = Prophet(
    changepoint_prior_scale=0.1,  # Increased trend flexibility
    seasonality_prior_scale=20.0, # Increased seasonal strength
    interval_width=0.75,
    yearly_seasonality= True,
    weekly_seasonality = True,
    daily_seasonality= False,
    holidays = holidays_df
)

In [47]:
m.add_regressor('Holiday_Flag')
m.fit(atm_data)

23:20:37 - cmdstanpy - INFO - Chain [1] start processing
23:20:37 - cmdstanpy - INFO - Chain [1] done processing


In [48]:
#Create Future Dates for Prediction 
future = m.make_future_dataframe(periods = 30, freq = 'D', include_history = False)

future['Holiday_Flag'] = 0 
forecast = m.predict(future)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Holiday_Flag,Holiday_Flag_lower,Holiday_Flag_upper,US_Bank_Holiday,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-12-26,44286.692414,41937.642383,81779.340395,44286.692414,44286.692414,0.0,0.0,0.0,0.000000,...,5183.972699,5183.972699,5183.972699,11414.746960,11414.746960,11414.746960,0.0,0.0,0.0,60885.412073
1,2023-12-27,44270.560292,34591.854320,77552.028494,44270.560292,44270.560292,0.0,0.0,0.0,0.000000,...,314.544660,314.544660,314.544660,11448.266372,11448.266372,11448.266372,0.0,0.0,0.0,56033.371324
2,2023-12-28,44254.428171,29078.577872,68982.879453,44254.428171,44254.428171,0.0,0.0,0.0,0.000000,...,-7597.355628,-7597.355628,-7597.355628,11333.980247,11333.980247,11333.980247,0.0,0.0,0.0,47991.052790
3,2023-12-29,44238.296049,39566.079676,81776.846030,44238.296049,44238.296049,0.0,0.0,0.0,0.000000,...,5683.803644,5683.803644,5683.803644,11066.120219,11066.120219,11066.120219,0.0,0.0,0.0,60988.219912
4,2023-12-30,44222.163927,31422.760181,72393.074794,44222.163927,44222.163927,0.0,0.0,0.0,0.000000,...,-2987.371867,-2987.371867,-2987.371867,10643.732853,10643.732853,10643.732853,0.0,0.0,0.0,51878.524913
5,2023-12-31,44206.031806,38528.901571,82809.084435,44206.031806,44206.031806,0.0,0.0,0.0,0.000000,...,5120.733550,5120.733550,5120.733550,10070.652538,10070.652538,10070.652538,0.0,0.0,0.0,59397.417894
6,2024-01-01,44189.899684,61576.108976,102596.696759,44189.899684,44189.899684,0.0,0.0,0.0,33003.727977,...,-5718.327057,-5718.327057,-5718.327057,9355.329278,9355.329278,9355.329278,0.0,0.0,0.0,80830.629882
7,2024-01-02,44173.767563,37506.661536,78130.117073,44173.767563,44173.767826,0.0,0.0,0.0,0.000000,...,5183.972699,5183.972699,5183.972699,8510.518968,8510.518968,8510.518968,0.0,0.0,0.0,57868.259229
8,2024-01-03,44157.635441,31475.391963,72125.356388,44157.635441,44157.636087,0.0,0.0,0.0,0.000000,...,314.544660,314.544660,314.544660,7552.847709,7552.847709,7552.847709,0.0,0.0,0.0,52025.027810
9,2024-01-04,44141.503320,25645.750746,63496.675665,44141.503204,44141.504405,0.0,0.0,0.0,0.000000,...,-7597.355628,-7597.355628,-7597.355628,6502.265336,6502.265336,6502.265336,0.0,0.0,0.0,43046.413027


In [54]:
import json
from prophet.serialize import model_to_json
# ... (Prophet model training code here) ...

# Assuming 'm' is your trained Prophet model object
model_filename = 'prophet_model_atm_0044.json'

with open(model_filename, 'w') as fout:
    # Serialize the model object into a JSON string and save it to the file
    json.dump(model_to_json(m), fout)

print(f"Model successfully saved as {model_filename}")

Model successfully saved as prophet_model_atm_0044.json
